# Lab For Experimentation

In [1]:
import json
import warnings
from typing import Any, Literal

import numpy as np
import pandas as pd
import polars as pl
from rich.console import Console
from rich.theme import Theme

custom_theme = Theme(
    {
        "white": "#FFFFFF",  # Bright white
        "info": "#00FF00",  # Bright green
        "warning": "#FFD700",  # Bright gold
        "error": "#FF1493",  # Deep pink
        "success": "#00FFFF",  # Cyan
        "highlight": "#FF4500",  # Orange-red
    }
)
console = Console(theme=custom_theme)

# Visualization
# import matplotlib.pyplot as plt

# NumPy settings
np.set_printoptions(precision=4)

# Pandas settings
pd.options.display.max_rows = 1_000
pd.options.display.max_columns = 1_000
pd.options.display.max_colwidth = 600

# Polars settings
pl.Config.set_fmt_str_lengths(1_000)
pl.Config.set_tbl_cols(n=1_000)
pl.Config.set_tbl_rows(n=200)

warnings.filterwarnings("ignore")

# Black code formatter (Optional)
%load_ext lab_black

# auto reload imports
%load_ext autoreload
%autoreload 2

In [2]:
def go_up_from_current_directory(*, go_up: int = 1) -> None:
    """This is used to up a number of directories.

    Params:
    -------
    go_up: int, default=1
        This indicates the number of times to go back up from the current directory.

    Returns:
    --------
    None
    """
    import os
    import sys

    CONST: str = "../"
    NUM: str = CONST * go_up

    # Goto the previous directory
    prev_directory = os.path.join(os.path.dirname(__name__), NUM)
    # Get the 'absolute path' of the previous directory
    abs_path_prev_directory = os.path.abspath(prev_directory)

    # Add the path to the System paths
    sys.path.insert(0, abs_path_prev_directory)
    print(abs_path_prev_directory)


# Demo (Prevents ruff from removing the unused module import)
name: Any
category: Literal["A", "B", "C"]
json.loads('{"name": "Bike Rental Prediction", "category": "A"}')

{'name': 'Bike Rental Prediction', 'category': 'A'}

In [3]:
go_up_from_current_directory(go_up=1)

/Users/mac/Desktop/Projects/Bike-Rental-Prediction


In [4]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split

In [5]:
rng = np.random.default_rng(123)
x = rng.standard_normal(size=(1_000, 10))

X_train, X_test = train_test_split(x, test_size=0.2, random_state=123)
y_train = rng.standard_normal(size=(X_train.shape[0],))
y_test = rng.standard_normal(size=(X_test.shape[0],))

params: dict[str, Any] = {
    "n_estimators": 100,
    "max_depth": 10,
}

rf_reg = RandomForestRegressor(**params)

rf_reg.fit(X_train, y_train)
# rf_reg.score(X_test, y_test)

,n_estimators,100
,criterion,'squared_error'
,max_depth,10
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,1.0
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


In [6]:
rf_reg.feature_importances_

array([0.1202, 0.1016, 0.0685, 0.1147, 0.0991, 0.1157, 0.0949, 0.0997,
       0.0957, 0.09  ])

In [9]:
from src.config import app_config
from src.exceptions import (
    CustomError,
    MLFlowConnectionError,
)
from src.ml.feature_engineering import FeatureEngineer
from src.ml.trainer import ModelTrainer

In [10]:
import httpx

port: int = 5001
url: str = f"http://localhost:{port}"


def check_mlflow(url: str, timeout: float = 2.0) -> bool:
    """
    Check MLflow endpoint accessibility and handle common httpx errors.

    Returns True if reachable (2xx), False otherwise.
    """
    try:
        resp = httpx.get(url, timeout=timeout)
        resp.raise_for_status()
        console.print("[success]MLflow is accessible")
        return True

    except httpx.HTTPStatusError as e:
        console.print(
            f"[error]MLflow returned non-2xx status: {e.response.status_code} — {e}"
        )
        return False

    except httpx.RequestError as e:
        # covers ConnectError, ReadTimeout, etc.
        console.print(f"[error]Network/connection error when contacting MLflow: {e}")
        return False

    except (MLFlowConnectionError, CustomError) as e:
        console.print(f"[error]Project-specific MLflow error: {e}")
        return False

    except Exception as e:
        console.print(f"[error]Unexpected error: {e}")
        return False


check_mlflow(url)

MLflow is accessible

True

In [11]:
fp: str = "../../../../Documents/data_dump/bike_data/database.parquet"
data: pl.DataFrame = pl.read_parquet(fp)
console.print(f"Shape: {data.shape}", style="info")

feat_eng = FeatureEngineer()
features_df: pl.DataFrame = feat_eng.create_all_features(
    data=data, config=app_config.feature_config
)
features_df.head()

Shape: (13903, 16)

season,mnth,hr,holiday,weekday,workingday,weathersit,temp,hum,is_weekend,sin_hour,cos_hour,sin_weekday,cos_weekday,cnt_lag_0hr,cnt_lag_1hr,cnt_lag_24hr,hr_lag_1hr,hr_lag_24hr,temp_lag_1hr,temp_lag_3hr,hum_lag_1hr,hum_lag_3hr,temp_rolling_mean_3hr,temp_rolling_median_3hr,temp_rolling_mean_6hr,temp_rolling_median_6hr,hum_rolling_mean_3hr,hum_rolling_median_3hr,hum_rolling_mean_6hr,hum_rolling_median_6hr,temp_plus_hum,hum_plus_hr,cnt_diff_1hr,cnt_diff_2hr,hr_diff_1hr,hr_diff_24hr,temp_diff_1hr,temp_diff_2hr,temp_diff_24hr,hum_diff_1hr,hum_diff_2hr,is_high_temp,is_high_hum,is_peak_hour,is_working_hour,is_business_hour,target
i64,i64,i64,i64,i64,i64,i64,f64,f64,i8,f64,f64,f64,f64,i64,i64,i64,i64,i64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i64,i64,i64,i64,f64,f64,f64,f64,f64,i8,i8,i8,i8,i8,i64
1,1,0,0,6,0,1,0.24,0.81,1,0.0,1.0,-0.781831,0.62349,16,16,16,0,0,0.24,0.24,0.81,0.81,0.226667,0.22,0.233333,0.24,0.803333,0.8,0.776667,0.775,1.05,0.81,24,16,1,0,-0.02,-0.02,0.22,-0.01,-0.01,0,0,0,0,0,40
1,1,1,0,6,0,1,0.22,0.8,1,0.258819,0.965926,-0.781831,0.62349,40,16,16,0,0,0.24,0.24,0.81,0.81,0.226667,0.22,0.233333,0.24,0.803333,0.8,0.776667,0.775,1.02,1.8,24,16,1,0,-0.02,-0.02,0.22,-0.01,-0.01,0,0,0,0,0,32
1,1,2,0,6,0,1,0.22,0.8,1,0.5,0.866025,-0.781831,0.62349,32,40,16,1,0,0.22,0.24,0.8,0.81,0.226667,0.22,0.233333,0.24,0.803333,0.8,0.776667,0.775,1.02,2.8,-8,16,1,0,0.0,-0.02,0.22,0.0,-0.01,0,0,0,0,0,13
1,1,3,0,6,0,1,0.24,0.75,1,0.707107,0.707107,-0.781831,0.62349,13,32,16,2,0,0.22,0.24,0.8,0.81,0.226667,0.22,0.233333,0.24,0.783333,0.8,0.776667,0.775,0.99,3.75,-19,-27,1,0,0.02,0.02,0.22,-0.05,-0.05,0,0,0,0,0,1
1,1,4,0,6,0,1,0.24,0.75,1,0.866025,0.5,-0.781831,0.62349,1,13,16,3,0,0.24,0.22,0.75,0.8,0.233333,0.24,0.233333,0.24,0.766667,0.75,0.776667,0.775,0.99,4.75,-12,-31,1,0,0.0,0.02,0.22,0.0,-0.05,0,0,0,0,0,1


In [ ]:
trainer = ModelTrainer(data=features_df)
trainer.hyperparameter_tuning_all_models()

In [12]:
import mlflow

mlflow.end_run()

In [13]:
console.print(app_config)

AppConfig(
    feature_config=FeatureConfig(
        lag_features=[
            Lags(feature='cnt', lags=[0, 1, 24]),
            Lags(feature='hr', lags=[1, 24]),
            Lags(feature='temp', lags=[1, 3]),
            Lags(feature='hum', lags=[1, 3])
        ],
        diff_features=[
            Diffs(feature='cnt', diffs=[1, 2]),
            Diffs(feature='hr', diffs=[1, 24]),
            Diffs(feature='temp', diffs=[1, 2, 24]),
            Diffs(feature='hum', diffs=[1, 2])
        ],
        interaction_features=[
            InteractionFeats(feature_1='temp', feature_2='hum', operation='add'),
            InteractionFeats(feature_1='hum', feature_2='hr', operation='add')
        ],
        rolling_features=[Windows(feature='temp', windows=[3, 6]), Windows(feature='hum', windows=[3, 6])],
        drop_features=['atemp', 'windspeed', 'casual', 'registered', 'datetime', 'cnt', 'yr'],
        target_col='cnt'
    ),
    experiment_config=ExperimentConfig(
        experiment_name='bike rental',
        experiment_tags=ExperimentTags(
            project='bike rental',
            metric_of_interest='RMSE',
            others={
                'team': 'data_science',
                'scientist': 'chinedu',
                'purpose': 'Hyperparameter tuning',
                'optimizer_engine': 'optuna',
                'feature_set_version': 1,
                'hardware': 'cpu'
            }
        )
    ),
    model_training_config=ModelTrainingConfig(
        general_config=GeneralConfig(random_seed=42, test_size=0.1, cv_test_size=168, n_splits=5),
        random_forest_config=RandomForestConfig(
            n_estimators=100,
            max_depth=10,
            min_samples_split=2,
            min_samples_leaf=1
        ),
        xgboost_config=XGBoostConfig(
            objective='reg:squarederror',
            n_estimators=100,
            learning_rate=0.1,
            early_stopping_rounds=20,
            num_boost_round=200
        ),
        lightgbm_config=LightGBMConfig(
            objective='regression',
            metric=['rmse', 'mae'],
            learning_rate=0.05,
            early_stopping_rounds=20,
            num_boost_round=200,
            verbose=1
        )
    ),
    optuna_config=OptunaConfig(
        random_forest_optuna_config=RandomForestOptunaConfig(
            n_estimators=(50, 300),
            max_depth=(3, 20),
            min_samples_split=(2, 10),
            min_samples_leaf=(1, 5),
            max_features=[None, 'sqrt', 'log2'],
            bootstrap=[True, False],
            n_trials=10
        ),
        xgboost_optuna_config=XGBoostOptunaConfig(
            objective='reg:squarederror',
            eval_metric='rmse',
            booster=['gbtree', 'gblinear', 'dart'],
            reg_lambda=(1e-06, 10.0),
            reg_alpha=(1e-06, 10.0),
            gamma=(1e-06, 10.0),
            max_depth=(2, 10),
            subsample=(0.1, 1.0),
            eta=(0.001, 0.6),
            n_estimators=(100, 2000),
            grow_policy=('depthwise', 'lossguide'),
            num_round=(100, 2000),
            early_stopping_rounds=(10, 100),
            n_trials=200
        ),
        lightgbm_optuna_config=LightGBMOptunaConfig(
            objective='regression',
            metric=['rmse', 'mae'],
            reg_lambda=(1e-06, 10.0),
            reg_alpha=(1e-06, 10.0),
            learning_rate=(0.01, 0.6),
            num_leaves=(20, 300),
            max_depth=(3, 15),
            min_child_samples=(5, 50),
            n_estimators=(100, 2000),
            n_trials=200,
            num_boost_round=(100, 2000),
            early_stopping_rounds=(10, 100)
        )
    ),
    api_config=APIConfig(
        title='Bike Rental Prediction API',
        name='Bike Rental Prediction API',
        description='API for predicting bike rental counts using a pre-trained model.',
        version='v0.0.1',
        status='healthy',
        prefix='/api/v1',
        server=Server(host='0.0